In [363]:
cd = {1:4, 2:6, 3:1, 4:8, 5:3, 10:2}

In [392]:
list = []
for key in cd.keys():
    for i in range(cd[key]):
        list.append(key)
list = sorted(list)

In [393]:
list

[202,
 221,
 234,
 240,
 245,
 251,
 251,
 252,
 254,
 255,
 256,
 258,
 258,
 259,
 259,
 263,
 263,
 264,
 266,
 267,
 268,
 269,
 269,
 270,
 271,
 273,
 273,
 274,
 274,
 275,
 276,
 276,
 277,
 278,
 278,
 278,
 280,
 280,
 280,
 280,
 280,
 281,
 282,
 282,
 283,
 283,
 284,
 284,
 285,
 285,
 286,
 287,
 288,
 288,
 288,
 288,
 288,
 289,
 290,
 291,
 291,
 292,
 293,
 294,
 294,
 294,
 294,
 294,
 295,
 295,
 295,
 296,
 296,
 297,
 297,
 297,
 298,
 298,
 298,
 298,
 298,
 299,
 299,
 300,
 300,
 300,
 300,
 301,
 301,
 302,
 303,
 304,
 304,
 304,
 304,
 304,
 304,
 304,
 304,
 305,
 305,
 305,
 305,
 306,
 307,
 307,
 307,
 307,
 307,
 309,
 309,
 309,
 309,
 309,
 309,
 310,
 311,
 312,
 312,
 312,
 313,
 313,
 313,
 314,
 315,
 315,
 315,
 316,
 317,
 318,
 318,
 319,
 320,
 320,
 320,
 321,
 321,
 322,
 322,
 323,
 324,
 324,
 325,
 326,
 327,
 327,
 330,
 330,
 330,
 331,
 331,
 331,
 331,
 331,
 331,
 332,
 332,
 332,
 333,
 334,
 334,
 335,
 335,
 336,
 337,
 340,
 342

In [369]:
def quick_median(count_dict: dict) -> int:
    """
    Finds the median of a counting dictionary
    :param count_dict: the counting dictionary to find the median of
    :return: integer index of the location of the median
    """
    list = []
    for key in count_dict.keys():
        for i in range(count_dict[key]):
            list.append(key)
    list = sorted(list)
    midpoint = len(list)//2
    if len(list)%2 == 0:
        median = (list[midpoint] + list[midpoint-1])//2
    else:
        median = list[midpoint]
    return median

In [305]:
def quick_median_old(count_dict: dict) -> int:
    mid_point = sum(count_dict.values()) / 2
    my_sum = 0
    my_ind = 0
    sk = sorted(count_dict.keys())
    while my_sum < mid_point:
        my_sum += count_dict[sk[my_ind]]
        if my_sum >= mid_point:
            break
        my_ind += 1
    return sk[my_ind]

In [323]:
cd = {4: 10, 236: 13, 99: 43, 88: 43, 127: 15}

In [370]:
quick_median(cd)

4

In [325]:
quick_median_old(cd)

99

In [326]:
cd2 = {4: 10, 236: 13, 99: 43, 88: 43, 127: 14, 8:102}

In [327]:
quick_median(cd2)

88

In [328]:
quick_median_old(cd2)

88

In [308]:
my_median1 = quick_median(cd)
my_median2 = quick_median(cd2)

In [309]:
def median_deviation_from_median(count_dict: dict) -> int:
    """
    calculates the deviation from the median of each element of counting dictionary,
    then returns the median of that dictionary
    :param count_dict: Counting dictionary to analyze
    :return: index of median of the deviations
    """
    my_median = quick_median(count_dict)
    deviations = {}
    for key in sorted(count_dict.keys()):
        X_value = abs(key - my_median)
        deviations[X_value] = count_dict[key]
    print(deviations)
    return quick_median(deviations)

In [342]:
median_deviation_from_median(cd)

{95: 10, 11: 43, 0: 43, 28: 15, 137: 13}


11

In [353]:
print(sorted([abs(x-99) for x in list]))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 137, 137, 137, 137, 137, 137, 137, 137, 137, 137, 137, 137, 137]


In [329]:
cd3 = {1:100, 100:100}

In [330]:
quick_median(cd3)

50

In [331]:
quick_median_old(cd3)

1

In [401]:
def quick_median_new(count_dict: dict) -> int:
    mid_point = sum(count_dict.values()) / 2
    my_sum = 0
    my_ind = 0
    sk = sorted(count_dict.keys())
    while my_sum < mid_point:
        my_sum += count_dict[sk[my_ind]]
        if my_sum >= mid_point:
            break
        my_ind += 1
    if sum(count_dict.values())%2 == 0:
        median = (sk[my_ind] + sk[my_ind-1])//2
    else:
        median = sk[my_ind]
    return median

In [378]:
quick_median_new(cd)

12.0
4
10
11
19


3

In [315]:
def count_frags(file: str) -> dict:
    """
    Takes a sam file input and creates a counting dictionary of the number of reads that are paired,
    first in the pair, confidently mapped and whose pair is mapped to the same reference
    :param file: A sam input file
    :return: A dictionary of the counts of the above reads
    """
    FILTER_MAPQUAL = 10  # only consider reads that are mapped with at least this mapping quality
    count_dict = {}
    PRINT_EVERY = 100000
    i = 0
    # Check if the file is sam or bam and decide how to open based on that
    if file[-4:] == ".sam":
        file_to_parse = open(file, 'r')
    elif file[-4:] == ".bam":
        file_to_parse = pysam.AlignmentFile(file, 'rb')
    else:
        print("Unknown file type, must be bam or sam")
        exit(1)

    for item in file_to_parse:
        # Need to convert bam iterable objects into strings for the next part
        line = str(item)
        # Skip all comments and headers
        if line[0] == '#' or line[0] == '@':
            continue
        splt = line.strip().split('\t')
        samFlag = int(splt[1])
        myRef = splt[2]
        mapQual = int(splt[4])
        mateRef = splt[6]
        myTlen = abs(int(splt[8]))

        # if read is paired, and is first in pair, and is confidently mapped...
        if samFlag & 1 and samFlag & 64 and mapQual > FILTER_MAPQUAL:
            # and mate is mapped to same reference
            if mateRef == '=' or mateRef == myRef:
                if myTlen not in count_dict:
                    count_dict[myTlen] = 0
                count_dict[myTlen] += 1
                i += 1
                if i % PRINT_EVERY == 0:
                    print('---', i, quick_median(count_dict), median_deviation_from_median(count_dict))
    return count_dict

In [391]:
cd = count_frags("C:/Users/joshf/Documents/work_stuff/neat_data/baby.sam")

In [317]:
FILTER_MAPQUAL = 10  # only consider reads that are mapped with at least this mapping quality
count_dict = {}
PRINT_EVERY = 100000
i = 0

In [24]:
for item in file_to_parse:
    # Need to convert bam iterable objects into strings for the next part
    line = str(item)
    # Skip all comments and headers
    if line[0] == '#' or line[0] == '@':
        continue
    splt = line.strip().split('\t')
    samFlag = int(splt[1])
    myRef = splt[2]
    mapQual = int(splt[4])
    mateRef = splt[6]
    myTlen = abs(int(splt[8]))

In [52]:
len(count_dict)

128

In [361]:
def compute_probs(count_dict: dict) -> (list, list):
    """
    Computes the probabilities for fragments with at least 100 pairs supporting it and that are at least 10 median
    deviations from the median.
    :param count_dict: A dictionary of fragments with counts
    :return: A list of values that meet the criteria and a list of -their associated probabilities
    """
    FILTER_MINREADS = 100  # only consider fragment lengths that have at least this many read pairs supporting it
    FILTER_MEDDEV_M = 10  # only consider fragment lengths this many median deviations above the median
    values = []
    probabilities = []
    med = quick_median(count_dict)
    mdm = median_deviation_from_median(count_dict)

    for key in sorted(count_dict.keys()):
        if 0 < key < med + FILTER_MEDDEV_M * mdm:
            if count_dict[key] >= FILTER_MINREADS:
                print(key, count_dict[key])
                values.append(key)
                probabilities.append(count_dict[key])
    count_sum = float(sum(probabilities))
    probabilities = [n / count_sum for n in probabilities]
    return values, probabilities

In [362]:
compute_probs(cd2)

{84: 10, 80: 102, 0: 43, 11: 43, 39: 14, 148: 13}
8 102


([8], [1.0])

In [394]:
import timeit

In [395]:
def wrapper(func, *args, **kwargs):
    def wrapped():
        return func(*args, **kwargs)
    return wrapped

In [404]:
import pysam
cd = count_frags("C:/Users/joshf/Documents/work_stuff/neat_data/WGS_chr20_21_22_normal.bam")

ModuleNotFoundError: No module named 'pysam'

In [397]:
wrapped = wrapper(quick_median, cd)

In [399]:
timeit.timeit(wrapped, number=1000)

0.06138989998726174

In [402]:
wrapped2 = wrapper(quick_median_new, cd)
timeit.timeit(wrapped2, number=1000)

0.01676309999311343